In [5]:
#Импорт моудлей
import pandas as pd
import datetime as dt


In [6]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.neighbors import KNeighborsClassifier

In [7]:
#Импорт данных
data = pd.read_csv('casc-resto.csv', sep=';', decimal='.', dtype='str')
dataConst = pd.read_csv('CASC_Constant.csv', sep=',', decimal='.', dtype='str')

In [8]:
#Привести колонку к типу данных datetime
data['RKDate'] = pd.to_datetime(data['RKDate'])

In [6]:
#Объявить константы
START_DATE = dt.datetime(2017, 7, 1)
END_DATE = dt.datetime(2017, 12, 31)
#Дата между
data['Visited'] = (data['RKDate'] <= END_DATE) & (data['RKDate'] > START_DATE)
customers = data.groupby(['CustomerID'])
# Пустой датафрейм
df = pd.DataFrame(columns=['CustomerID', 'Visited', 'Recency', 'Frequency', 'Monetary Value', 'Mean Saved'])

In [13]:
#Перебор групп customerid
for state, frame in customers:
    dates = frame[frame['RKDate'] < START_DATE]
    if not dates.empty:
        #Дни между полседним визитом и 2017.07.01
        recency = START_DATE - max(dates['RKDate'])
        recency = recency.days
        #частота
        frequency = dates.shape[0] / ((START_DATE - min(dates['RKDate'])).days/30)
        #Средняя сумма
        monetaryValue = dates['SummBasic'].mean()
        #Средняя сумма сохраненных денег
        meanSaved = (dates['SummBasic'] - dates['SummAfterPointsUsage']).mean()
        #Добавить строку в датафрейм
        df = df.append(
            {'CustomerID': state, 'Visited': int(frame['Visited'].any()), 'Recency': recency, 'Frequency': frequency,
             'Monetary Value': monetaryValue, 'Mean Saved': meanSaved}, ignore_index=True)

In [14]:
#Изменить имя колонки
dataConst.rename(columns={'CustomerId': 'CustomerID'}, inplace=True)
#Объеденить датафреймы
merged = pd.merge(df, dataConst, on='CustomerID')
#Удлить ненужное
merged = merged.drop(columns=['ActivationDate', 'SubscribedEmail', 'SubscribedPush'])
merged = merged.dropna()

In [15]:
#Привести к инт
merged['Visited'] = merged['Visited'].astype(int)
y = merged['Visited']
merged = merged.drop(columns=['Visited'])
#Привести к инт
merged['Age'] = merged['Age'].astype(int)
merged['Sex'] = (merged['Sex'] == 'Female').astype(int)

In [16]:
#Разделить выборки на тестовую и тренировочную
X_train, X_test, y_train, y_test = train_test_split(merged, y, test_size=0.2)
#Экземпляр LogReg
logreg = LogisticRegression()

In [17]:
#Подставить данные
logreg.fit(X_train, y_train)
#Предасказать y
y_pred = logreg.predict(X_test)
print(y_pred)

[1 0 1 ... 1 1 1]


In [18]:
#Оценка качетва предсказания
scoreTrain = logreg.score(X_train, y_train)
print(scoreTrain)
score = logreg.score(X_test, y_test)
print(score)

0.7355113636363636
0.7392045454545455


In [19]:
# оценка показателей precision и recall
knn = KNeighborsClassifier(n_neighbors=8)
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.70      0.73      0.72       935
           1       0.68      0.64      0.66       825

    accuracy                           0.69      1760
   macro avg       0.69      0.69      0.69      1760
weighted avg       0.69      0.69      0.69      1760

